In [ ]:
#Import
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

In [ ]:
#kaggle
!pip install -q kaggle

In [ ]:
#Upload kaggle api
from google.colab import files
files.upload()

In [ ]:
#Create directory
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Download dataset and unzip
!kaggle datasets download -d techsash/waste-classification-data
!unzip waste-classification-data.zip

In [ ]:
#Set train and test directory
base_dir="/content/DATASET"
train_dir=os.path.join(base_dir,"TRAIN")
test_dir=os.path.join(base_dir,"TEST")

In [ ]:
#Prepare dataset
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True)

validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(150,150),
                                                  batch_size=64,
                                                  class_mode='binary')

validation_generator=validation_datagen.flow_from_directory(test_dir,
                                                            target_size=(150,150),
                                                            batch_size=64,
                                                            class_mode='binary')

In [ ]:
#Model
#Aku ganti last layer pake softmax sama nambah Conv2D+MaxPooling2D satu
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(2)
    ])

In [ ]:
#Callback dengan akurasi 85 persen
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self,epoch,logs={}):
        if((logs.get('accuracy')>0.85) and (logs.get('val_accuracy')>0.85)):
          self.model.stop_training = True

In [ ]:
#Train model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
model.fit(train_generator,steps_per_epoch=25,epochs=50,validation_data=validation_generator,validation_steps=5,verbose=2,callbacks=myCallback())

In [ ]:
#Create TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

In [ ]:
#Save TFLite model
#Download manual di foldernya
with open("model.tflite", "wb") as f:
    f.write(tflite_model)